# Metropolis algorithm

## What is autocorrelation time?

Autocorrelation time quantifies how long a system remembers its past — in other words, how many steps it takes for correlations between successive states to decay. In computational physics and statistical mechanics, this concept is essential for understanding how efficiently an algorithm samples the system’s equilibrium distribution.

There are two main ways to estimate the autocorrelation time:

1. Theoretically (heuristically) — by analyzing the physical processes or algorithmic mechanisms that govern relaxation (e.g., diffusion, collective motion, or energy barriers). This helps identify what slows down convergence but may miss hidden slow modes, leading to an underestimate.

2. Empirically — by computing the autocorrelation function of several physical observables (like energy, magnetization, or particle density). If none of these observables couple strongly to the slowest mode, the true autocorrelation time may again be underestimated. Therefore: there
is always the danger that our **chosen set of observables has failed to include one that has strong enough overlap with the slowest mode leading to an underestimate**.

The integrated autocorrelation time, often denoted $\tau_{\text{int}}$, determines the effective number of independent samples collected. Physically, it corresponds to how fast the system relaxes toward equilibrium — thus linking the algorithm’s sampling efficiency to actual dynamical or collective phenomena in the model. For example, near a critical point, long-range correlations in physical quantities manifest as very long autocorrelation times, a phenomenon known as **critical slowing down**.

The actual rate of convergence to equilibrium from a given initial distribution may be much faster than the worst-case estimate given by $\tau_{\text{exp}}$. Thus, it is usual to determine empirically when "equilibrium" has been achieved, by plotting selected observables as a function of time and noting when the initial transient appears to end.

Watch out for **metastability**!
For example, near a first-order phase transition, most Monte Carlo methods suffer from metastability associated with transitions between configurations typical of the distinct pure phases. We can try initial conditions typical of each of these phases (e.g., for many models, a "hot" start (random initial configuration) and a "cold" start (ordered initial start; all spins up or all spins down)). Consistency between these runs does not guarantee
that metastability is absent, but it does give increased confidence. Plots of observables as a function of time are also useful indicators of possible metastability.


**Once we estimate how long it takes to reach equilibrium**, we simply **discard (“burn in”) the initial portion of the data** — up to a cutoff time $n_{\text{disc}}$.

Although this is not strictly necessary in the asymptotic limit (since systematic errors from the transient decay faster than statistical ones), in practice, keeping non-equilibrium data can introduce significant bias if the starting configuration is far from equilibrium. Discarding the transient data therefore helps ensure that only equilibrium samples contribute to averages, improving the accuracy of estimated physical quantities.

Even in **finite-size lattices**, a cutoff is required. The only difference is that, since the system is finite, its equilibration and correlation times are in principle bounded. Still, one must determine an appropriate thermalization time for each lattice size (often a few times the autocorrelation time of the slowest mode) before starting data collection.

For the 2D Ising model, we discard approximately $20\tau$ of initial data to remove initialization bias and run for at least $1000\tau$ to ensure percent-level statistical accuracy. These ratios hold in general, though the autocorrelation time $\tau$ increases strongly near the critical temperature due to critical slowing down and depends on both lattice size and simulation algorithm.

## Comparison of algorithms

The better algorithm is the one that has the smaller autocorrelation time, when time is measured in units of computer (CPU) time.

## Implementation of algorithm for 2D Ising model

In [1]:
import random

In [2]:
def metropolis_algorithm(initial_state, transition_function, acceptance_function, num_iterations):
    """
    Perform the Metropolis algorithm for sampling from a probability distribution.

    Parameters:
    initial_state: The starting state of the system.
    transition_function: A function that proposes a new state given the current state.
    acceptance_function: A function that computes the acceptance probability of the proposed state.
    num_iterations: The number of iterations to perform.

    Returns:
    A list of sampled states.
    """
    current_state = initial_state
    samples = [current_state]

    for _ in range(num_iterations):
        proposed_state = transition_function(current_state)
        acceptance_prob = acceptance_function(current_state, proposed_state)

        if random.random() < acceptance_prob:
            current_state = proposed_state

        samples.append(current_state)

    return samples

In [4]:
def Hamiltonian_2D_Ising_with_external_field(state, J, h):
    """
    Calculate the Hamiltonian of a 2D Ising model with an external magnetic field.

    Parameters:
    state: A 2D list representing the spin configuration (+1 or -1).
    J: Interaction strength between neighboring spins.
    h: External magnetic field strength.

    Returns:
    The Hamiltonian value.
    """
    H = 0
    rows = len(state)
    cols = len(state[0])

    for i in range(rows):
        for j in range(cols):
            # Interaction with right neighbor
            H -= J * state[i][j] * state[i][(j + 1) % cols]
            # Interaction with bottom neighbor
            H -= J * state[i][j] * state[(i + 1) % rows][j]
            # Interaction with external magnetic field
            H -= h * state[i][j]

    return H